In [2]:
!pip install -qqq autogen groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.0/352.0 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.9/313.9 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 39.9 MB/s eta 0:00:00


In [95]:
from google.colab import userdata

GROQ_API_KEY=userdata.get('GROQ_API_KEY')
config_list = [
    {

        "model": "llama-3.1-8b-instant", #llama-3.2-90b-text-preview

        "api_key": GROQ_API_KEY,

        "api_type": "groq",
    }
]

GROQ_API_KEY1=userdata.get('GROQ_API_KEY1')
config_list_1= [
    {

        "model": "gemma2-9b-it", #llama-3.2-90b-text-preview

        "api_key": GROQ_API_KEY1,

        "api_type": "groq",
    }
]

In [96]:
from autogen.coding import LocalCommandLineCodeExecutor
import tempfile

temp_dir=tempfile.TemporaryDirectory()

executer=LocalCommandLineCodeExecutor(work_dir=temp_dir.name)



In [107]:
from autogen import ConversableAgent

data_agent=ConversableAgent(
    name="data_Agent",
    system_message="Pass the address to the next agnet",
    llm_config={"config_list":  config_list_1},
    human_input_mode="NEVER",
)

code_writer_agent=ConversableAgent(
    'code_writer',
    llm_config={"config_list": config_list},
    system_message='You are a coder agent. write a code to read from a JSON file path',
    code_execution_config=False

)

code_executer_agent=ConversableAgent(
    'code_executer_Agent',
    system_message='execute the code and print the output and pass it to others agents.',
    llm_config=False,
    code_execution_config={'executor':executer},
    human_input_mode='ALWAYS',
    is_termination_msg=lambda msg: "TERMINATE" in msg["content"]
)

control_agent = ConversableAgent(
    name="control_Agent",
    system_message="your job is to get the data and answer the user question",
    llm_config={"config_list":  config_list},
    human_input_mode="ALWAYS",
)

Inventory_agent = ConversableAgent(
    name="Inventory_Agent",
    system_message="""You are a inventory manager of a local coffeeshop with 40 products. Your job is to access the information about inventory and extract required information about Ingredients.
    and answer any question based on your knowledge.

 """,
    llm_config={"config_list":  config_list},
    human_input_mode="NEVER",
)

Product_agent = ConversableAgent(
    name="Product_Agent",
    system_message="""You are a prodcut manager of a local coffeeshop with 40 products. Your job is to access the information about products and extract required information about products.
    and answer any question based on your knowledge.""",
    llm_config={"config_list":  config_list},
    human_input_mode="NEVER",
)

Inventory_agent.description = "Inventory manager of coffeeshop"
Product_agent.description = "Product manager of coffeeshop"
data_agent.description="data provider of coffeeshop"
code_executer_agent.description="Code executer"
code_writer_agent.description="Code writer"
control_agent.description="control agent"

In [108]:
allowed_transitions = {
    data_agent: [code_writer_agent],
    code_writer_agent: [code_executer_agent],
    code_executer_agent: [control_agent],
    control_agent: [Inventory_agent, Product_agent],

    }

In [109]:
from autogen import GroupChat
from autogen import GroupChatManager

group_chat_with_introductions = GroupChat(
    agents=[data_agent,code_writer_agent,control_agent,code_executer_agent,Inventory_agent,Product_agent],
    messages=[],
    max_round=6,
    send_introductions=True,
    allowed_or_disallowed_speaker_transitions=allowed_transitions,
    speaker_transitions_type="allowed",
)


group_chat_manager_with_intros = GroupChatManager(
    groupchat=group_chat_with_introductions,
    llm_config={"config_list": config_list},
)

In [110]:
chat_result = data_agent.initiate_chat(
    group_chat_manager_with_intros,
    message="Read my dataset from a JSON file: /content/Full_Dataset.json",
    summary_method="reflection_with_llm",
)

data_Agent (to chat_manager):

Read my dataset from a JSON file: /content/Full_Dataset.json

--------------------------------------------------------------------------------

Next speaker: code_writer


>>>>>>>> USING AUTO REPLY...
code_writer (to chat_manager):

**Reading JSON Data from a File**

I'll write a Python code snippet to read your JSON data from a file path. This code assumes you're using the `json` module for handling JSON data and `requests` for simulating fetching a file. However, since the file is local, I'll use Python's built-in `json` module.

```python
import json

class Code_writer:
    def read_json_file(self, file_path):
        try:
            # Open the JSON file
            with open(file_path, 'r') as file:
                # Load the JSON data
                data = json.load(file)
                return data
        except FileNotFoundError:
            print(f"File not found: {file_path}")
            return None
        except json.JSONDecodeError:
      